In [0]:
%run /Workspace/Users/aminah.yussuf@slalom.com/Includes/Copy-Datasets

In [0]:
%python
files = dbutils.fs.ls("dbfs:/mnt/demo-datasets/bookstore/books-csv")
display(files)

In [0]:
SELECT * FROM JSON.`dbfs:/mnt/demo-datasets/bookstore/customers-json`

In [0]:
SELECT *, input_file_name() sourcefile
FROM JSON.`dbfs:/mnt/demo-datasets/bookstore/customers-json`

In [0]:
CREATE TABLE IF NOT EXISTS books_csv
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS ( 
  header = "true", 
  delimiter = ";"
)
LOCATION "dbfs:/mnt/demo-datasets/bookstore/books-csv";


select * from books_csv

In [0]:
DESCRIBE EXTENDED books_csv;

In [0]:
%python 
#this command will read data from the books csv table and write it back into the source csv file 
(spark.read
   .table("books_csv")
   .write
        .mode("overwrite")
        .format("csv")
        .option("header", "true")
        .option("delimiter", ';')
        .save("dbfs:/mnt/demo-datasets/bookstore/books_csv"))

In [0]:
REFRESH TABLE books_csv;

SELECT COUNT(*) FROM books_csv;

In [0]:
CREATE TABLE IF NOT EXISTS customers AS 
SELECT * FROM JSON.`dbfs:/mnt/demo-datasets/bookstore/customers-json`;

DESCRIBE EXTENDED customers;

In [0]:
CREATE TABLE IF NOT EXISTS orders AS
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders`;

SELECT * FROM orders;

In [0]:
----create a table from a parquest file 
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders`;

SELECT * FROM orders;

DESCRIBE HISTORY orders;

In [0]:
----insert overwrite, overwrite does not create a new table, it basically replaces the data in the same table with selected data
INSERT OVERWRITE orders
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders`;

In [0]:
----creaye a temporary view from a csv file with a predefined schema (schema is not auto generated from csv file) 
CREATE OR REPLACE TEMP VIEW books_updates 
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV 
OPTIONS (
  path = "dbfs:/mnt/demo-datasets/bookstore/books-csv-new",
  header = "true",
  delimiter = ";"
);

SELECT * FROM books_updates;

In [0]:
SELECT * FROM customers;

In [0]:
----create a temporary view and trasverse the profile json object using the ":"

SELECT customer_id, profile:first_name, profile:address:country  FROM customers

In [0]:

-----create a temperary view to parse the json with the retrieved schema above 
CREATE OR REPLACE TEMP VIEW parsed_customers AS
    SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Dniren","last_name":"Abby","gender":"Female","address":{"street":"768 Mesta Terrace","city":"Annecy","country":"France"}}')) profile_struct
FROM customers;

---describe customer table 
DESCRIBE TABLE customers;

---select all from customers table
SELECT * FROM customers;

In [0]:

--create a temporary view of with json element split into it own columns 
CREATE OR REPLACE TEMP VIEW customers_final AS 
SELECT customer_id, profile_struct.* FROM parsed_customers;

In [0]:
SELECT * FROM customers_final

In [0]:
SELECT order_id, customer_id, books from orders

In [0]:
---use the "explode" keyword to seperate elements in an array of columns into its own row 
SELECT order_id, customer_id, explode(books) AS book
FROM orders

In [0]:
---use "collect_set" keyword to aggregate order id as key and book id per order as the corresponding array
SELECT customer_id, 
    collect_set(order_id) AS order_set,
    collect_set(books.book_id) AS books_set
FROM orders
GROUP BY customer_id

In [0]:
---join operation
----standard join operation includes inner, outter, left, right, full and cross 

CREATE OR REPLACE VIEW orders_enriched AS
SELECT * 
FROM 
    (SELECT *, explode(books) AS book
    FROM orders) o
    INNER JOIN books_csv b
    ON o.book.book_id = b.book_id;

----SELECT * FROM orders_enriched;

DESCRIBE TABLE orders_enriched;

In [0]:
-----create a table using the pivot function to create new column with sum of quantity per book item purchased by customer
CREATE OR REPLACE TABLE transactions AS 

SELECT * FROM ( 
  SELECT 
    customer_id, 
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (
  sum(quantity) FOR book_id in (
    'B01','BO2','B03','BO4','B05','BO6', 
    'B07','BO8','B09','B10','B11','B12') 
);

SELECT * FROM transactions;